# CEAPS Data Clean up

This project aims to analyze/generate insights over the CEAPS(Cota para Exercício da Atividade Parlamentar dos Senadores - Quota for the Exercise of Parliamentary Activity by Senators) data, collected from 2008 to 2022.

In order to achieve that, such data needs to be cleaned up. This file works on that.

**Notes**: 
- When trying to import the csv's, got an error saying: 'utf-8' codec can't decode byte 0xe7 in position 205: invalid continuation byte.
Using the linux terminal command "file <file_name>", found that the encoding of the file is on ISO-8859.
- Separators are semicoluns.
- Needs to skip one line at the start of the file.

In [52]:
import pandas as pd
import numpy as np
import datetime as dt

Libraries versions:

In [2]:
print ("Pandas: %s" % pd.__version__)
print ("Numpy: %s" % np.__version__)

Pandas: 2.2.2
Numpy: 1.26.4


## Building the DataFrame

Here I check if all data files have the same columns.

In [3]:
expected_columns = ['ANO', 'MES', 'SENADOR', 'TIPO_DESPESA', 'CNPJ_CPF', 'FORNECEDOR', 'DOCUMENTO', 'DATA', 'DETALHAMENTO', 'VALOR_REEMBOLSADO', 'COD_DOCUMENTO']
columns_np_array = np.array(expected_columns)
amount_of_equals = 0
years_array = range(2008, 2023, 1)
years_with_different_columns = []

# Commenting out as is unnecessary on subsequent executions.
#for i in years_array:
#    dt = pd.read_csv(f'despesa_ceaps_{i}.csv', encoding='ISO-8859-1', sep=';', skiprows=1)
#    if np.array_equal(dt.columns, columns_np_array):
#        amount_of_equals += 1
#    else:
#        years_with_different_columns.append(i)

#if amount_of_equals == len(years_array):
#    print("All files have the same columns and shapes.")
#else:
#    print("Not all files have the same columns and shapes. The following years are different: ")
#    print(years_with_different_columns)

Considering all files have the same columns and shapes, I can bring them all to a single DataFrame.

In [4]:
main_df = pd.DataFrame(columns=expected_columns)
for i in years_array:
    dt = pd.read_csv(f'despesa_ceaps_{i}.csv', encoding='ISO-8859-1', sep=';', skiprows=1)
    main_df = pd.concat([main_df, dt], ignore_index=True)

main_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317984 entries, 0 to 317983
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ANO                317984 non-null  object
 1   MES                317984 non-null  object
 2   SENADOR            317984 non-null  object
 3   TIPO_DESPESA       317984 non-null  object
 4   CNPJ_CPF           314840 non-null  object
 5   FORNECEDOR         314840 non-null  object
 6   DOCUMENTO          293174 non-null  object
 7   DATA               314840 non-null  object
 8   DETALHAMENTO       180631 non-null  object
 9   VALOR_REEMBOLSADO  317984 non-null  object
 10  COD_DOCUMENTO      317982 non-null  object
dtypes: object(11)
memory usage: 26.7+ MB


In [5]:
main_df.head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2008,9,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,"12351,52",2008090814004
1,2008,9,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,"386,6",2008090814003
2,2008,10,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,"12351,52",2008100814004
3,2008,10,ADA MELLO,"Locomoção, hospedagem, alimentação, combustíve...",NaN,NaN,NaN,NaN,NaN,"2610,68",2008100814003
4,2008,11,ADA MELLO,"Contratação de consultorias, assessorias, pesq...",NaN,NaN,NaN,NaN,NaN,"12351,52",2008110814004


In [6]:
len(main_df.SENADOR.unique())

291

The first impression is that might not be valuable to remove rows with null values on certain columns from the dataset.

Observe that although we have null values on CNPJ/CPF, FORNECEDOR, DOCUMENTO, DATA and DETALHAMENTO, we always have values on ANO, MES, SENADOR, TIPO_DESPESA, VALOR_REEMBOLSADO e COD_DOCUMENTO. That allows to at least calculate the Senator's expenses, although, without many details for some of them.

## Cleaning up expense types

In [7]:
main_df.TIPO_DESPESA.unique()

array(['Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar',
       'Locomoção, hospedagem, alimentação, combustíveis e lubrificantes',
       'Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos. ',
       'Divulgação da atividade parlamentar',
       'Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.',
       'Passagens aéreas, aquáticas e terrestres nacionais',
       'Serviços de Segurança Privada'], dtype=object)

Going to map the expenses types to numbers in order to make it easy to generate visualizations later on.

In [8]:
expenses_mapping  = {
    'Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar': 1,
    'Locomoção, hospedagem, alimentação, combustíveis e lubrificantes': 2,
    'Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos. ': 3,
    'Divulgação da atividade parlamentar': 4,
    'Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.': 5,
    'Passagens aéreas, aquáticas e terrestres nacionais': 6,
    'Serviços de Segurança Privada': 7
}

In [9]:
main_df['TIPO_DESPESA'] = main_df['TIPO_DESPESA'].map(expenses_mapping)
main_df.head(10)

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2008,9,ADA MELLO,1,NaN,NaN,NaN,NaN,NaN,"12351,52",2008090814004
1,2008,9,ADA MELLO,2,NaN,NaN,NaN,NaN,NaN,"386,6",2008090814003
2,2008,10,ADA MELLO,1,NaN,NaN,NaN,NaN,NaN,"12351,52",2008100814004
3,2008,10,ADA MELLO,2,NaN,NaN,NaN,NaN,NaN,"2610,68",2008100814003
4,2008,11,ADA MELLO,1,NaN,NaN,NaN,NaN,NaN,"12351,52",2008110814004
5,2008,11,ADA MELLO,2,NaN,NaN,NaN,NaN,NaN,"2615,63",2008110814003
6,2008,12,ADA MELLO,1,NaN,NaN,NaN,NaN,NaN,"12351,52",2008120814004
7,2008,12,ADA MELLO,2,NaN,NaN,NaN,NaN,NaN,"4943,47",2008120814003
8,2008,2,ADELMIR SANTANA,1,NaN,NaN,NaN,NaN,NaN,13800,2008020738104
9,2008,2,ADELMIR SANTANA,2,NaN,NaN,NaN,NaN,NaN,"1151,13",2008020738103


## Cleaning up CNPJ/CPF

In [10]:
main_df[main_df['CNPJ_CPF'].notna()].head()

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
2449,2009,12,ACIR GURGACZ,5,004.948.028-63,GILBERTO PISELO DO NASCIMENTO,NaN,11/12/2009,NaN,5000,222467
2450,2009,12,ACIR GURGACZ,4,02.831.112/0002-09,INTERCOM INTERMEDIAÇÕES E COMUNICAÇÃO INTEGRAD...,000330,09/12/2009,NaN,12620,222466
2456,2009,4,ADELMIR SANTANA,2,00.306.597/0016-83,Cascol combustíveis para veículos Ltda,106471,12/04/2009,NaN,"179,01",165740
2457,2009,4,ADELMIR SANTANA,2,00.358.192/0001-02,Ribeiro e Pereira Ltda,77472,04/04/2009,NaN,30,165739
2458,2009,4,ADELMIR SANTANA,2,06.098.111/0001-69,Tudo de Bom Comércio de alimentos Ltda,10169,14/04/2009,NaN,"215,8",165741


We can see that CNPJ_CPF column values may contain special characters like '.', '-' and '/'.

Going to remove those characters so to transform the column into a numeric type.

In [11]:
main_df['CNPJ_CPF'] = main_df['CNPJ_CPF'].apply(lambda x: x.replace('.', '').replace('-', '').replace('/', '').strip() if pd.notna(x) else x)

In [12]:
main_df['CNPJ_CPF'] = main_df['CNPJ_CPF'].apply(lambda x: x if pd.notna(x) else 0)

In [13]:
main_df['CNPJ_CPF'] = main_df['CNPJ_CPF'].astype('int64')


In [14]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317984 entries, 0 to 317983
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ANO                317984 non-null  object
 1   MES                317984 non-null  object
 2   SENADOR            317984 non-null  object
 3   TIPO_DESPESA       317984 non-null  int64 
 4   CNPJ_CPF           317984 non-null  int64 
 5   FORNECEDOR         314840 non-null  object
 6   DOCUMENTO          293174 non-null  object
 7   DATA               314840 non-null  object
 8   DETALHAMENTO       180631 non-null  object
 9   VALOR_REEMBOLSADO  317984 non-null  object
 10  COD_DOCUMENTO      317982 non-null  object
dtypes: int64(2), object(9)
memory usage: 26.7+ MB


## Cleaning up suppliers

Needs to adjust casing on the suppliers columns to ease the reading, turn it into string and adjust the NaN values.

In [15]:
main_df['FORNECEDOR'] = main_df['FORNECEDOR'].apply(lambda x: x.title() if pd.notna(x) else x)

In [16]:
main_df['FORNECEDOR'] = main_df['FORNECEDOR'].astype(str)

In [17]:
main_df

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2008,9,ADA MELLO,1,0,nan,NaN,NaN,NaN,"12351,52",2008090814004
1,2008,9,ADA MELLO,2,0,nan,NaN,NaN,NaN,"386,6",2008090814003
2,2008,10,ADA MELLO,1,0,nan,NaN,NaN,NaN,"12351,52",2008100814004
3,2008,10,ADA MELLO,2,0,nan,NaN,NaN,NaN,"2610,68",2008100814003
4,2008,11,ADA MELLO,1,0,nan,NaN,NaN,NaN,"12351,52",2008110814004
...,...,...,...,...,...,...,...,...,...,...,...
317979,2022,12,ZEQUINHA MARINHO,6,22052777000132,Exceller Tour,WIXHAI,06/12/2022,"Companhia Aérea: LATAM, Localizador: WIXHAI. P...","2893,04",2191398
317980,2022,12,ZEQUINHA MARINHO,6,22052777000132,Exceller Tour,WITOLM,09/12/2022,"Companhia Aérea: GOL, Localizador: WITOLM. Pas...","1180,19",2192272
317981,2022,12,ZEQUINHA MARINHO,6,22052777000132,Exceller Tour,THPKVQ,20/12/2022,"Companhia Aérea: TAM, Localizador: THPKVQ. Pas...","2671,9",2192274
317982,2022,12,ZEQUINHA MARINHO,6,22052777000132,Exceller Tour,QNN9HX,21/12/2022,"Companhia Aérea: AZUL, Localizador: QNN9HX. Pa...","1334,31",2192244


In [18]:
main_df['FORNECEDOR'] = main_df['FORNECEDOR'].replace('nan', 'NaN')

In [19]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317984 entries, 0 to 317983
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ANO                317984 non-null  object
 1   MES                317984 non-null  object
 2   SENADOR            317984 non-null  object
 3   TIPO_DESPESA       317984 non-null  int64 
 4   CNPJ_CPF           317984 non-null  int64 
 5   FORNECEDOR         317984 non-null  object
 6   DOCUMENTO          293174 non-null  object
 7   DATA               314840 non-null  object
 8   DETALHAMENTO       180631 non-null  object
 9   VALOR_REEMBOLSADO  317984 non-null  object
 10  COD_DOCUMENTO      317982 non-null  object
dtypes: int64(2), object(9)
memory usage: 26.7+ MB


## Cleaning up senators

Here I will just adjust the name's casing.

In [20]:
main_df['SENADOR'] = main_df['SENADOR'].apply(lambda x: x.title())
main_df['SENADOR']

0                Ada Mello
1                Ada Mello
2                Ada Mello
3                Ada Mello
4                Ada Mello
                ...       
317979    Zequinha Marinho
317980    Zequinha Marinho
317981    Zequinha Marinho
317982    Zequinha Marinho
317983    Zequinha Marinho
Name: SENADOR, Length: 317984, dtype: object

## Cleaning up dates and documents

Needs to convert existing dates to a date type.

First will check if all dates have the same format.

In [21]:
not_formatted_dates = main_df[~main_df['DATA'].str.match(r'^[0-9]{2}/[0-9]{2}/[0-9]{4}$', na=False)]
not_formatted_dates

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2008,9,Ada Mello,1,0,NaN,NaN,NaN,NaN,"12351,52",2008090814004
1,2008,9,Ada Mello,2,0,NaN,NaN,NaN,NaN,"386,6",2008090814003
2,2008,10,Ada Mello,1,0,NaN,NaN,NaN,NaN,"12351,52",2008100814004
3,2008,10,Ada Mello,2,0,NaN,NaN,NaN,NaN,"2610,68",2008100814003
4,2008,11,Ada Mello,1,0,NaN,NaN,NaN,NaN,"12351,52",2008110814004
...,...,...,...,...,...,...,...,...,...,...,...
18403,2009,3,Wellington Salgado De Oliveira,3,0,NaN,NaN,NaN,NaN,"599,56",2009030587702
18404,2009,3,Wellington Salgado De Oliveira,1,0,NaN,NaN,NaN,NaN,3480,2009030587704
18405,2009,3,Wellington Salgado De Oliveira,2,0,NaN,NaN,NaN,NaN,"4261,92",2009030587703
150213,2015,8,Eduardo Amorim,6,13353495000184,Propagtur - Propag Turismo Ltda,"SEM FATURA"";02/08/2015""","Companhia Aérea: TAM, Localizador: YXGDSJ. Pas...","469,53",2014675,NaN


We see that there are two rows on the DataFrame that didn't parse correctly on DOCUMENTO column and forward. Needs to adjust that.

In [22]:
main_df.iloc[150213]['DATA']

'Companhia Aérea: TAM, Localizador: YXGDSJ. Passageiros: JOSÉ SILVIO MONTEIRO (Matrícula 188223, COMISSIONADO), Voo: 3538 - BRASÍLIA/ARACAJU - 07/08/2015;'

In [23]:
main_df.iloc[150220]['DATA']

'Companhia Aérea: AVIANCA, Localizador: ZNEU9F. Passageiros: EDUARDO ALVES DO AMORIM (Matrícula 234312, PARLAMENTAR), Voo: 6223 - ARACAJU/BRASÍLIA - 24/08/2015;'

In [24]:
main_df.loc[150213, 'DOCUMENTO'] = 'NaN'
main_df.loc[150213, 'DATA'] = '02/08/2015'
main_df.loc[150213, 'DETALHAMENTO'] = 'Companhia Aérea: TAM, Localizador: YXGDSJ. Passageiros: JOSÉ SILVIO MONTEIRO (Matrícula 188223, COMISSIONADO), Voo: 3538 - BRASÍLIA/ARACAJU - 07/08/2015;'
main_df.loc[150213, 'VALOR_REEMBOLSADO'] = '469,53'
main_df.loc[150213, 'COD_DOCUMENTO'] = '2014675'

main_df.loc[150220, 'DOCUMENTO'] = 'NaN'
main_df.loc[150220, 'DATA'] = '17/08/2015'
main_df.loc[150220, 'DETALHAMENTO'] = 'Companhia Aérea: AVIANCA, Localizador: ZNEU9F. Passageiros: EDUARDO ALVES DO AMORIM (Matrícula 234312, PARLAMENTAR), Voo: 6223 - ARACAJU/BRASÍLIA - 24/08/2015;'
main_df.loc[150220, 'VALOR_REEMBOLSADO'] = '460,45'
main_df.loc[150220, 'COD_DOCUMENTO'] = '2016557'

main_df.iloc[[150213, 150220]]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
150213,2015,8,Eduardo Amorim,6,13353495000184,Propagtur - Propag Turismo Ltda,NaN,02/08/2015,"Companhia Aérea: TAM, Localizador: YXGDSJ. Pas...","469,53",2014675
150220,2015,8,Eduardo Amorim,6,13353495000184,Propagtur - Propag Turismo Ltda,NaN,17/08/2015,"Companhia Aérea: AVIANCA, Localizador: ZNEU9F....","460,45",2016557


Although had some warnings on the updates above, if we check the out of format rows again, it should only display the NaNs:

In [25]:
not_formatted_dates = main_df[~main_df['DATA'].str.match(r'^[0-9]{2}/[0-9]{2}/[0-9]{4}$', na=False)]
not_formatted_dates

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO
0,2008,9,Ada Mello,1,0,NaN,NaN,NaN,NaN,"12351,52",2008090814004
1,2008,9,Ada Mello,2,0,NaN,NaN,NaN,NaN,"386,6",2008090814003
2,2008,10,Ada Mello,1,0,NaN,NaN,NaN,NaN,"12351,52",2008100814004
3,2008,10,Ada Mello,2,0,NaN,NaN,NaN,NaN,"2610,68",2008100814003
4,2008,11,Ada Mello,1,0,NaN,NaN,NaN,NaN,"12351,52",2008110814004
...,...,...,...,...,...,...,...,...,...,...,...
18401,2009,2,Wellington Salgado De Oliveira,2,0,NaN,NaN,NaN,NaN,"3058,13",2009020587703
18402,2009,3,Wellington Salgado De Oliveira,5,0,NaN,NaN,NaN,NaN,"2988,72",2009030587701
18403,2009,3,Wellington Salgado De Oliveira,3,0,NaN,NaN,NaN,NaN,"599,56",2009030587702
18404,2009,3,Wellington Salgado De Oliveira,1,0,NaN,NaN,NaN,NaN,3480,2009030587704


Now we convert documents to string.

In [26]:
main_df['DOCUMENTO'] = main_df['DOCUMENTO'].apply(lambda x: x if pd.notna(x) else 'NaN')

In [27]:
main_df['FORNECEDOR'] = main_df['FORNECEDOR'].astype(str)
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317984 entries, 0 to 317983
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   ANO                317984 non-null  object
 1   MES                317984 non-null  object
 2   SENADOR            317984 non-null  object
 3   TIPO_DESPESA       317984 non-null  int64 
 4   CNPJ_CPF           317984 non-null  int64 
 5   FORNECEDOR         317984 non-null  object
 6   DOCUMENTO          317984 non-null  object
 7   DATA               314840 non-null  object
 8   DETALHAMENTO       180631 non-null  object
 9   VALOR_REEMBOLSADO  317984 non-null  object
 10  COD_DOCUMENTO      317984 non-null  object
dtypes: int64(2), object(9)
memory usage: 26.7+ MB


Now if we tried to convert dates to a date format again, we would get errors like the following: 'Out of bounds nanosecond timestamp: 06/10/2915, at position 2596.'.

Tried some shots to check if it was just some of the data that were wrong:

In [28]:
main_df.loc[154649, 'DATA'] = '06/10/2015'

Trying again we spot other problems like: Out of bounds nanosecond timestamp: 26/08/0201, at position 2598.

So just going to fix them:

In [29]:
main_df.loc[156117, 'DATA'] = '26/08/2015'
main_df.loc[156470, 'DATA'] = '24/02/2015'
main_df.loc[162852, 'DATA'] = '22/04/2015'
main_df.loc[162856, 'DATA'] = '23/04/2015'
main_df.loc[167257, 'DATA'] = '08/05/2015'
main_df.loc[172538, 'DATA'] = '20/07/2016'

As it seemed too much effort, decided to convert the column to DateTime using 'errors=coerce' and have it on a new column, then we can identify the rows with NaT in it and see how many are still broken.

In [30]:
main_df['Converted_DATA'] = pd.to_datetime(main_df['DATA'], exact=True, format='%d/%m/%Y', errors='coerce')

In [31]:
main_df[main_df['Converted_DATA'].isna() & ~main_df['DATA'].isna()]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO,Converted_DATA
182785,2016,4,José Medeiros,2,12262251000123,Hotel Gran Odara Ltda,65509,02/04/3016,despesa com hospedagem,"259,9",2034257,NaT
183965,2016,4,José Serra,2,52791571000195,A Ferreira Automóveis Ltda,541327,04/04/0216,Senador José Serra - Aquisição de combustível ...,"158,58",2036344,NaT
184080,2016,10,Kátia Abreu,2,56548779000139,Transamérica Comercial E Serviços Ltda Scp La ...,00038600,31/10/0200,"Hospedagem (R$ 648,44) e alimentação (R$ 40,00...","688,44",2051549,NaT
188325,2016,10,Paulo Paim,6,33937681000178,Latam,5HJBB4,31/10/0206,"Companhia Aérea: LATAM, Localizador: 5HJBB4. P...","1860,79",2051216,NaT
203728,2017,12,Elmano Férrer,5,6277571000154,Imobiliária Santa Clara Ltda.,73859,06/02/1017,DESPESA COM PAGAMENTO DE ALUGUEL DO ESCRITÓRIO...,"2973,45",2083148,NaT
212862,2017,11,Lúcia Vânia,2,2072286000650,Petroil Combustívels Ltda.,5021,30/11/1201,Aquisição de combustível para veículo utilizad...,100,2084741,NaT
213093,2017,11,Magno Malta,5,2558157000162,Telefonica Brasil S.A.,05162707630,09/11/0201,Despesa com imóvel político: pagamento de tele...,"421,28",2084537,NaT
215642,2017,3,Paulo Rocha,6,7575651000159,Vrg Linhas Aereas S.A.,PKIP6Q,02/03/0201,"Companhia Aérea: GOL, Localizador: PKIP6Q. Pas...","731,79",2060535,NaT
223560,2017,3,Zezé Perrella,6,33937681000178,Latam,3WI7U2,13/02/0217,"Companhia Aérea: LATAM, Localizador: 3WI7U2. P...","430,51",2062578,NaT
238079,2018,3,Lúcia Vânia,5,2266468000158,Urbs Imobiliária,177528,09/05/0208,Pagamento de taxa de aluguel referente ao imóv...,1200,2096075,NaT


As it seems feasible, decided to ajust the invalid DATA rows so to not lose data.

In [32]:
main_df.loc[182785, 'DATA'] = '02/04/2016'
main_df.loc[183965, 'DATA'] = '04/04/2016'
main_df.loc[184080, 'DATA'] = '31/10/2016'
main_df.loc[188325, 'DATA'] = '31/10/2016'
main_df.loc[203728, 'DATA'] = '06/12/2017'
main_df.loc[212862, 'DATA'] = '30/11/2017'
main_df.loc[213093, 'DATA'] = '09/11/2017'
main_df.loc[215642, 'DATA'] = '02/03/2017'
main_df.loc[223560, 'DATA'] = '13/02/2017'
main_df.loc[238079, 'DATA'] = '09/03/2018'
main_df.loc[240697, 'DATA'] = '16/11/2018'
main_df.loc[259616, 'DATA'] = '31/01/2019'
main_df.loc[280626, 'DATA'] = '05/05/2020'
main_df.loc[286502, 'DATA'] = '29/10/2021'

Now we convert the original DATA column to datetime and get rid of the temporary one.

In [33]:
main_df['DATA'] = pd.to_datetime(main_df['DATA'], exact=True, format='%d/%m/%Y')

In [34]:
main_df.drop('Converted_DATA', axis=1, inplace=True)

Let's fill the remaining NaN values on the dates column based on the Month and Year columns values.

In [56]:
def fill_dates(row):
    if pd.notna(row['DATA']):
        return row['DATA']
    else:
        return pd.to_datetime(f'10/{row["MES"]}/{row["ANO"]}', format='%d/%m/%Y')

In [60]:
main_df['DATA'] = main_df.apply(fill_dates, axis=1)

In [61]:
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317984 entries, 0 to 317983
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ANO                317984 non-null  int64         
 1   MES                317984 non-null  int64         
 2   SENADOR            317984 non-null  object        
 3   TIPO_DESPESA       317984 non-null  int64         
 4   CNPJ_CPF           317984 non-null  int64         
 5   FORNECEDOR         317984 non-null  object        
 6   DOCUMENTO          317984 non-null  object        
 7   DATA               317984 non-null  datetime64[ns]
 8   DETALHAMENTO       317984 non-null  object        
 9   VALOR_REEMBOLSADO  317984 non-null  float32       
 10  COD_DOCUMENTO      317984 non-null  int64         
dtypes: datetime64[ns](1), float32(1), int64(5), object(4)
memory usage: 25.5+ MB


## Cleaning up details

In [36]:
main_df['DETALHAMENTO'] = main_df['DETALHAMENTO'].apply(lambda x: x if pd.notna(x) else 'NaN')

In [37]:
main_df['DETALHAMENTO'] = main_df['DETALHAMENTO'].astype(str)
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317984 entries, 0 to 317983
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ANO                317984 non-null  object        
 1   MES                317984 non-null  object        
 2   SENADOR            317984 non-null  object        
 3   TIPO_DESPESA       317984 non-null  int64         
 4   CNPJ_CPF           317984 non-null  int64         
 5   FORNECEDOR         317984 non-null  object        
 6   DOCUMENTO          317984 non-null  object        
 7   DATA               314840 non-null  datetime64[ns]
 8   DETALHAMENTO       317984 non-null  object        
 9   VALOR_REEMBOLSADO  317984 non-null  object        
 10  COD_DOCUMENTO      317984 non-null  object        
dtypes: datetime64[ns](1), int64(2), object(8)
memory usage: 26.7+ MB


## Cleaning up document codes

In [38]:
main_df['COD_DOCUMENTO'] = main_df['COD_DOCUMENTO'].astype('int64')
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317984 entries, 0 to 317983
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ANO                317984 non-null  object        
 1   MES                317984 non-null  object        
 2   SENADOR            317984 non-null  object        
 3   TIPO_DESPESA       317984 non-null  int64         
 4   CNPJ_CPF           317984 non-null  int64         
 5   FORNECEDOR         317984 non-null  object        
 6   DOCUMENTO          317984 non-null  object        
 7   DATA               314840 non-null  datetime64[ns]
 8   DETALHAMENTO       317984 non-null  object        
 9   VALOR_REEMBOLSADO  317984 non-null  object        
 10  COD_DOCUMENTO      317984 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(7)
memory usage: 26.7+ MB


## Cleaning up refunded values

In [41]:
main_df[main_df['VALOR_REEMBOLSADO'].str.contains('.', regex=False)]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO


In [42]:
main_df[main_df['VALOR_REEMBOLSADO'].str.contains(' ', regex=False)]

,ANO,MES,SENADOR,TIPO_DESPESA,CNPJ_CPF,FORNECEDOR,DOCUMENTO,DATA,DETALHAMENTO,VALOR_REEMBOLSADO,COD_DOCUMENTO


Apparently the values are well formatted using comma as decimal separator.

In [46]:
main_df['VALOR_REEMBOLSADO'] = main_df['VALOR_REEMBOLSADO'].str.replace(',', '.', regex=False).replace(r'[^0-9.]', '', regex=True)
main_df['VALOR_REEMBOLSADO'].head(10)

0    12351.52
1       386.6
2    12351.52
3     2610.68
4    12351.52
5     2615.63
6    12351.52
7     4943.47
8       13800
9     1151.13
Name: VALOR_REEMBOLSADO, dtype: object

In [47]:
main_df['VALOR_REEMBOLSADO'] = main_df['VALOR_REEMBOLSADO'].astype(np.float32)
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317984 entries, 0 to 317983
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ANO                317984 non-null  object        
 1   MES                317984 non-null  object        
 2   SENADOR            317984 non-null  object        
 3   TIPO_DESPESA       317984 non-null  int64         
 4   CNPJ_CPF           317984 non-null  int64         
 5   FORNECEDOR         317984 non-null  object        
 6   DOCUMENTO          317984 non-null  object        
 7   DATA               314840 non-null  datetime64[ns]
 8   DETALHAMENTO       317984 non-null  object        
 9   VALOR_REEMBOLSADO  317984 non-null  float32       
 10  COD_DOCUMENTO      317984 non-null  int64         
dtypes: datetime64[ns](1), float32(1), int64(3), object(6)
memory usage: 25.5+ MB


## Cleaning up year and month

In [48]:
main_df['ANO'] = main_df['ANO'].astype('int64')
main_df['MES'] = main_df['MES'].astype('int64')
main_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317984 entries, 0 to 317983
Data columns (total 11 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   ANO                317984 non-null  int64         
 1   MES                317984 non-null  int64         
 2   SENADOR            317984 non-null  object        
 3   TIPO_DESPESA       317984 non-null  int64         
 4   CNPJ_CPF           317984 non-null  int64         
 5   FORNECEDOR         317984 non-null  object        
 6   DOCUMENTO          317984 non-null  object        
 7   DATA               314840 non-null  datetime64[ns]
 8   DETALHAMENTO       317984 non-null  object        
 9   VALOR_REEMBOLSADO  317984 non-null  float32       
 10  COD_DOCUMENTO      317984 non-null  int64         
dtypes: datetime64[ns](1), float32(1), int64(5), object(4)
memory usage: 25.5+ MB


## Checking for duplicates

In [65]:
main_df['COD_DOCUMENTO'].duplicated().value_counts()

COD_DOCUMENTO
False    317984
Name: count, dtype: int64

As no apparent duplicates, no need to remove any rows.

## Exporting formatted data

Now that we have the values cleaned up, we can export it back to a csv file that is ready for importing and analisis.

In [67]:
main_df.to_csv('./despesa_ceaps_2008_2022_formatted.csv', encoding='utf-8', sep=';', date_format='%d/%m/%Y', index=False)